In [5]:
# !pip install --no-deps --force-reinstall snowflake_ml_python-1.0.12-py3-none-any.whl

In [11]:
# !pip install peft transformers tokenizers vllm bitsandbytes datasets absl-py

### Import Libraries

In [10]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
import snowflake.ml.playground.finetune as ft_util
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Establish Secure Connection

In [13]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.39.4
Snowpark for Python version : 1.9.0


### Reference Llama 2 from Huggingface

In [14]:
options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)
llama_model = llm.LLM(
    model_id_or_path="meta-llama/Llama-2-7b-chat-hf",
    options=options
)

### Register and Deploy Llama 2 into Snowpark Container Services 

*NOTE: Logging and deploying the same model are one time operations. Once the model is logged and deployed, use ModeReference to get the reference to the model.*

In [15]:
MODEL_NAME    = 'LLAMA2_7b_CHAT'
MODEL_VERSION = "1.0.2"
MODEL_REGISTRY_DB  = "DASH_DB"
MODEL_REGISTRY_SCHEMA  = "DASH_SCHEMA"
COMPUTE_POOL  = "DASH_BUILD_GPU3"

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

# llama_model_ref = registry.log_model(
#     model_name=MODEL_NAME,
#     model_version=MODEL_VERSION,
#     model=llama_model
# )

# llama_model_ref.deploy(
#     deployment_name="llama_predict", 
#     platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
#     permanent=True, 
#     options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

llama_model_ref = model_registry.ModelReference(registry=registry,model_name=MODEL_NAME,model_version=MODEL_VERSION)
llama_model_ref

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


### Load Data from JSON into Snowflake

*NOTE: Reading data in JSON and storing it in a Snowflake table are one time operations. Once the data is loaded, use Snowpark to load the data from the existing table.*

In [16]:
# df = pd.read_json("../day1/data/frosty_transcripts_all_v8.json",lines=True)
# sf_df = session.write_pandas(df,'frosty_transcripts',auto_create_table=True,quote_identifiers=False)

sf_df = session.table('frosty_transcripts')
sf_df.show(max_width=600)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LANGUAGE"  |"TRANSCRIPT"                                                                                                                                                                               |"NAME"  |"LOCATION"  |"TOY_LIST"                                                                |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|EN          |caller: Hello!                                                                        

### Prompt Engineering Example

In [17]:
begin_prompt = \
"""
[INST] Summarize this transcript in less than 200 words: 
"""
end_prompt = " [/INST]"

df_inputs = sf_df.with_column('"input"',F.concat_ws(F.lit(" "),F.lit(begin_prompt),F.col('transcript'),F.lit(end_prompt))).select('"input"')
df_inputs.show(max_width=600)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"input"                                                                                                                                                                                    |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|                                                                                                                                                                                           |
|[INST] Summarize this transcript in less than 200 words:                                                                                                                                   |
| caller: Hello!                                  

### Inference using Custom Prompt

In [18]:
df_predict_results = llama_model_ref.predict(deployment_name="llama_predict",data=df_inputs)
df_predict_results.select('"generated_text"').show(max_width=600)

/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:310: RuntimeWarning: Warn in feature input: Nullable column "input" provided, inference might fail if there is null value.
  warnings.warn(


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"generated_text"                                                                                                                                                                                                                                                                                                                                                |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
!mkdir merged_dir
!mkdir output_dir

In [36]:
df = pd.read_json("train.json", lines=True).head(10)

merged_dir = 'merged_dir'
output_dir = 'output_dir'

runner = ft_util.FinetuneModelRunner(
    base_model_id="meta-llama/Llama-2-7b-chat-hf",
    task_type=ft_util.FinetuneTaskType.INSTRUCT,
    train_dataset=df,
    eval_dataset=df,
)

mcf = ft_util.ModelConfig(
    max_steps=1,
    quantization=ft_util.QuantizationConfig(
        k_bits=4,
    ),
    merged_weights_dir_path=merged_dir,
)

_ = runner.train(output_dir, model_cfg=mcf)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Map (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO 11-09 22:21:11 llm_engine.py:72] Initializing an LLM engine with config: model='merged_dir', tokenizer='merged_dir', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 11-09 22:22:01 llm_engine.py:207] # GPU blocks: 864, # CPU blocks: 512


In [38]:
eval_df = runner.get_eval_results()
eval_df

Processed prompts: 100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


,instruction,input,output,language,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hello!\nfrosty: Well, hello! Who's spreading holiday cheer with me today?\ncaller: I'm Max from Sydney.\nfrosty: Hello, Max! Can you tell me what's on your wish list this holiday?\ncaller: Hmm, I am not sure. I guess I like cars.\nfrosty: We have a fun Bluey car. It's very cool. And also, there's a Teenage Mutant Ninja Turtles pizza delivery van! It's really fun.\ncaller: Oh, the bluey car sounds cool.\nfrosty: Great choice, Max! By the way, how do you plan to celebrate the holiday season with your family?\ncaller: We're going to the beach! It's summer here in Sydney.\nfrosty: Oh, that sounds wonderful, Max. So, we will put the Bluey car on your holiday wish list, okay?\ncaller: Yes, please!\nfrosty: It\u2019s all done. I hope your holiday is filled with joy and fun!""","{""toy_list"": [""Bluey Convertible and Figures""], ""location"": ""Sydney""}",EN,"Here is the JSON response with ""location"" and ""toy_list"" as keys:\n\n{\n""location"": ""Sydney"",\n""toy_list"": [\n""Bluey car"",\n""Teenage Mutant Ninja Turtles pizza delivery van""\n]\n}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi, Frosty\nfrosty: Hello there, bundle of joy! Can you share your name?\ncaller: I'm Lily.\nfrosty: Hello, Lily! And where are you calling from?\ncaller: I'm from London.\nfrosty: Beautiful London! Now, Lily, what are your two most wished toys this holiday season?\ncaller: I think I want the furby and... or maybe the transformers bumblebee. I can't decide.\nfrosty: Both are fascinating choices, Lily! Why are you interested in these toys?\ncaller: I like furby because it can talk, and transformers bumblebee because I like robots.\nfrosty: How about we put both on your wish list for now and you can decide later?\ncaller: Yes, let's do that.\nfrosty: Alright, Lily, both toys are now on your wish list. Enjoy your holiday and stay joyful!""","{""toy_list"": [""Furby interactive plush toy"", ""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""London"",\n""toy_list"": [\n""furby"",\n""transformers bumblebee""\n]\n}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi Frosty! It's Benny here.\nfrosty: Hello, Benny! Nice to hear from you. Where are you calling from?\ncaller: From Auckland.\nfrosty: Beautiful Auckland! Now, tell Frosty your holiday wish, Benny.\ncaller: Can I have the ninja turtles delivery van?\nfrosty: Of course, you can. By the way, Benny, what do you enjoy the most about this time of the year?\ncaller: I like when we light up our house with beautiful lights.\nfrosty: That sounds like a sparkling time! So, I'll put the ninja turtles delivery van on your wish list. Does that sound good?\ncaller: Yes, that's perfect!\nfrosty: I wish you the best and super fun holidays, Benny!""","{""toy_list"": [""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""], ""location"": ""Auckland""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Auckland"",\n""toy_list"": [\n""ninja turtles delivery van""\n]\n}"
3,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hello there! I'm Frosty, how can I add some sparkle to your holiday season?\ncaller: Hi Frosty! I'm Billy from Denver, United States.\nfrosty: Hi Billy! It's great to hear from you. What brings you to me today?\ncaller: Well I'm trying to decide whi

In [39]:
runner.infer(
    {
    'instruction': "Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.",
    'input': """
"frosty: Hello, hello! Who's ready to chat with Frosty today?\ncaller: Hi, I'm Dustin from Cape Town, South Africa.\nfrosty: Hi Dustin, it's awesome to meet you! Do you already know what you would like for your holiday gift?\ncaller: Yeah, I think I want the ninja turtles delivery van, or maybe the robot dog. I can't decide...\nfrosty: Both sound like entertaining choices! Why are you interested in these particular toys?\ncaller: The ninja turtles van is so cool! And the robot dog, it's like a pet you don't have to clean up after.\nfrosty: That's true! Savage choices, Dustin! How do you usually celebrate the holiday season?\ncaller: We usually have a big family barbeque and play games all day long.\nfrosty: Sounds fun! What is your favorite thing about this time of year?\ncaller: I like that everyone is happy and I get to spend a lot of time with my family.\nfrosty: That's wonderful! So Dustin, have you thought any more about your holiday wish?\ncaller: I think I'm going to get the robot dog. It sounds fun and I won't have to walk it.\nfrosty: That's a clever choice, Dustin! I hope you have a fantastic holiday season!"
"""
    }
)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


'Here is the JSON response with the requested keys:\n\n{\n"location": "Cape Town, South Africa",\n"toy_list": ["ninja turtles delivery van", "robot dog"]\n}'

In [42]:
test_res = runner.infer_batch(df)
test_res

Processed prompts: 100%|██████████| 10/10 [00:04<00:00,  2.23it/s]


,instruction,input,output,language,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hello!\nfrosty: Well, hello! Who's spreading holiday cheer with me today?\ncaller: I'm Max from Sydney.\nfrosty: Hello, Max! Can you tell me what's on your wish list this holiday?\ncaller: Hmm, I am not sure. I guess I like cars.\nfrosty: We have a fun Bluey car. It's very cool. And also, there's a Teenage Mutant Ninja Turtles pizza delivery van! It's really fun.\ncaller: Oh, the bluey car sounds cool.\nfrosty: Great choice, Max! By the way, how do you plan to celebrate the holiday season with your family?\ncaller: We're going to the beach! It's summer here in Sydney.\nfrosty: Oh, that sounds wonderful, Max. So, we will put the Bluey car on your holiday wish list, okay?\ncaller: Yes, please!\nfrosty: It\u2019s all done. I hope your holiday is filled with joy and fun!""","{""toy_list"": [""Bluey Convertible and Figures""], ""location"": ""Sydney""}",EN,"Here is the JSON response with ""location"" and ""toy_list"" as keys:\n\n{\n""location"": ""Sydney"",\n""toy_list"": [\n""Bluey car"",\n""Teenage Mutant Ninja Turtles pizza delivery van""\n]\n}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi, Frosty\nfrosty: Hello there, bundle of joy! Can you share your name?\ncaller: I'm Lily.\nfrosty: Hello, Lily! And where are you calling from?\ncaller: I'm from London.\nfrosty: Beautiful London! Now, Lily, what are your two most wished toys this holiday season?\ncaller: I think I want the furby and... or maybe the transformers bumblebee. I can't decide.\nfrosty: Both are fascinating choices, Lily! Why are you interested in these toys?\ncaller: I like furby because it can talk, and transformers bumblebee because I like robots.\nfrosty: How about we put both on your wish list for now and you can decide later?\ncaller: Yes, let's do that.\nfrosty: Alright, Lily, both toys are now on your wish list. Enjoy your holiday and stay joyful!""","{""toy_list"": [""Furby interactive plush toy"", ""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""London"",\n""toy_list"": [\n""furby"",\n""transformers bumblebee""\n]\n}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi Frosty! It's Benny here.\nfrosty: Hello, Benny! Nice to hear from you. Where are you calling from?\ncaller: From Auckland.\nfrosty: Beautiful Auckland! Now, tell Frosty your holiday wish, Benny.\ncaller: Can I have the ninja turtles delivery van?\nfrosty: Of course, you can. By the way, Benny, what do you enjoy the most about this time of the year?\ncaller: I like when we light up our house with beautiful lights.\nfrosty: That sounds like a sparkling time! So, I'll put the ninja turtles delivery van on your wish list. Does that sound good?\ncaller: Yes, that's perfect!\nfrosty: I wish you the best and super fun holidays, Benny!""","{""toy_list"": [""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""], ""location"": ""Auckland""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Auckland"",\n""toy_list"": [\n""ninja turtles delivery van""\n]\n}"
3,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hello there! I'm Frosty, how can I add some sparkle to your holiday season?\ncaller: Hi Frosty! I'm Billy from Denver, United States.\nfrosty: Hi Billy! It's great to hear from you. What brings you to me today?\ncaller: Well I'm trying to decide whi

In [46]:
# NOTE: This will take ~20mins

MODEL_VERSION='1.0.3'

options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path=output_dir,
    options=options
)

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name="llama_predict", 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/_internal/env_utils.py:217: UserWarning: Package requirement absl-py<2,>=0.15 specified, while version 2.0.0 is installed. Local version will be ignored to conform to package requirement.
  warnings.warn(
/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


{'name': 'DASH_DB.DASH_SCHEMA.llama_predict',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_BUILD_GPU3', 'num_gpus': 1},
 'details': {'image_name': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/8a1453d3c1d39172488042f22e4a323c912464a3:latest',
  'service_spec': "spec:\n  container:\n  - env:\n      MODEL_ZIP_STAGE_PATH: /DASH_DB.DASH_SCHEMA.SNOWML_MODEL_F19D13E67F5211EE8E8F4615E23FEC97/model.zip\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer\n      _CONCURRENT_REQUESTS_MAX: 1\n    image: sfsenort